In [184]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/
import json

dataset = pd.read_json('NER_TRAIN_JUDGEMENT.json')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks


In [185]:
len(dataset)

9435

In [186]:
dataset = dataset.drop(['meta'], axis=1)

In [187]:
dataset['annotations'][0][0]['result']

[{'value': {'start': 90,
   'end': 103,
   'text': 'Hongkong Bank',
   'labels': ['ORG']},
  'id': 'C8HPTIM1',
  'from_name': 'label',
  'to_name': 'text',
  'type': 'labels'},
 {'value': {'start': 267,
   'end': 278,
   'text': 'Rahul & Co.',
   'labels': ['ORG']},
  'id': 'KOWE3RAM',
  'from_name': 'label',
  'to_name': 'text',
  'type': 'labels'}]

In [188]:
extract_text = lambda x: x['text']
dataset['data'] = dataset['data'].apply(extract_text)

In [189]:
dataset

,id,annotations,data
0,90d9a97c7b7749ec8a4f460fda6f937e,"[{'result': [{'value': {'start': 90, 'end': 10...",\n\n(7) On specific query by the Bench about a...
1,a325c57ba5b84c6fa46bee65e6616633,"[{'result': [{'value': {'start': 26, 'end': 30...","He was also asked whether Agya <span class=""hi..."
2,33cface31108441080976035ee04d07e,"[{'result': [{'value': {'start': 13, 'end': 25...","\n5.2 CW3 Mr Vijay Mishra , Deputy Manager, H..."
3,08debe900b51464e9f26b5d1eecae170,[{'result': []}],You are hereby asked not to carry out any cons...
4,428070222cd940eba78b233829eb28b8,"[{'result': [{'value': {'start': 18, 'end': 43...",The pillion rider T.V. Satyanarayana Murthy al...
...,...,...,...
9430,5cb430a6d0e14e57a968f0c43d219682,"[{'result': [{'value': {'start': 202, 'end': 2...",It is prayed in the application that the suit ...
9431,6e9147c9e6344865a8b095fb7f43f7c2,"[{'result': [{'value': {'start': 26, 'end': 35...","In the first instance, Mr.A.D.Desai contended ..."
9432,657ef5df817d4e95a36d702b4e41ce7d,"[{'result': [{'value': {'start': 0, 'end': 5, ...","Sikri, J. (as he then was), speaking for the m..."
9433,9ff4b03ee09f49a6814b9550c145ea1e,"[{'result': [{'value': {'start': 1, 'end': 17,...",(RAJENDRA MAHAJAN) JUDGE AKM M.Cr.C. No. 8763/...


In [190]:
import re
dataset['data'] = dataset['data'].apply(lambda x: re.sub(' +', ' ', x))
dataset['tokens'] = dataset['data'].apply(lambda x: x.split())

In [191]:
entity_list = ["OTHERS", "PETITIONER", "COURT", "RESPONDENT", "JUDGE", "OTHER_PERSON", "LAWYER", "DATE", "ORG", "GPE", "STATUTE", "PROVISION", "PRECEDENT", "CASE_NUMBER", "WITNESS"]
label_encoding_dict = {"OTHERS": 0 , "PETITIONER": 1, "COURT": 2, "RESPONDENT": 3, "JUDGE": 4, "OTHER_PERSON": 5, "LAWYER": 6, "DATE": 7, "ORG": 8, "GPE": 9, "STATUTE": 10, "PROVISION": 11, "PRECEDENT": 12, "CASE_NUMBER": 13, "WITNESS": 14}

In [192]:
greeting = ['hello','my','name','is','bob','how','are','you','my','name','is']
def find_sub_list(sublist, original_list):
    sublist_len=len(sublist)
    for ind in (i for i,e in enumerate(original_list) if e==sublist[0]):
        if original_list[ind:ind+sublist_len]==sublist:
            return ind,ind+sublist_len-1

find_sub_list(['name'], greeting)

(2, 2)

In [193]:
tokens = ['In',
 'the',
 'first',
 'instance,',
 'Mr.A.D.Desai',
 'contended',
 'that',
 'the',
 'petitioners',
 'might',
 'not',
 'have',
 'led',
 'full',
 'evidence',
 'before',
 'the',
 'District',
 'Court,',
 'knowing',
 'or',
 'at',
 'least',
 'believing',
 'that',
 'the',
 'proceedings',
 'before',
 'the',
 'District',
 'Court',
 'were',
 'illegal',
 'by',
 'reason',
 'of',
 'non-compliance',
 'with',
 'the',
 'requirements',
 'of',
 'Section',
 '56', 'B',
 'and',
 'that',
 'if',
 'that',
 'be',
 'the',
 'position,',
 'the',
 'petitioners',
 'would',
 'be',
 'considerably',
 'prejudiced',
 'by',
 'refusal',
 'to',
 'interfere',
 'with',
 'the',
 'order',
 'of',
 'the',
 'District',
 'Court',
 'in',
 'revision.']
sentence = " ".join(tokens)
print(sentence)
entity = {"value": {"start": 23, "end": 25, "text": "Mr.", "labels": ["PER"]}}

start_ind=sentence.find(entity["value"]["text"])
if sentence[start_ind-1]!=" ":
  start_index = len(sentence[:entity["value"]["start"]].split())
  i=start_index-1
  st = tokens[i]
  j=st.find(entity["value"]["text"])
  tokens[i:i+1] = [tokens[i][:j], tokens[i][j:]]

end_ind=sentence.find(entity["value"]["text"]) + len(entity["value"]["text"]) - 1
if sentence[end_ind+1]!=" ":
  print("END")
  end_index = len(sentence[:entity["value"]["end"]].split())
  i=end_index-1
  st = tokens[i]
  j=st.find(entity["value"]["text"])+len(entity["value"]["text"])
  tokens[i:i+1] = [tokens[i][:j], tokens[i][j:]]



ner_tags = [0] * len(tokens)
ent = entity["value"]["text"].split(" ")
print(ent)
print(tokens)
start_index, end_index = find_sub_list(ent, tokens)
end_index+=1
print(start_index, end_index)
print(tokens)

# Split the entity text into separate tokens if it is not already split
entity_tokens = entity["value"]["text"].split()

# Set the NER tag to 1 for tokens within the start and end indices if the label is "PER"
for i in range(len(tokens)):
    if i >= start_index and i < end_index and "PER" in entity["value"]["labels"]:
        ner_tags[i] = 1

print(tokens[start_index: end_index])
print(ner_tags)

In the first instance, Mr.A.D.Desai contended that the petitioners might not have led full evidence before the District Court, knowing or at least believing that the proceedings before the District Court were illegal by reason of non-compliance with the requirements of Section 56 B and that if that be the position, the petitioners would be considerably prejudiced by refusal to interfere with the order of the District Court in revision.
END
['Mr.']
['In', 'the', 'first', 'instance,', 'Mr.', 'A.D.Desai', 'contended', 'that', 'the', 'petitioners', 'might', 'not', 'have', 'led', 'full', 'evidence', 'before', 'the', 'District', 'Court,', 'knowing', 'or', 'at', 'least', 'believing', 'that', 'the', 'proceedings', 'before', 'the', 'District', 'Court', 'were', 'illegal', 'by', 'reason', 'of', 'non-compliance', 'with', 'the', 'requirements', 'of', 'Section', '56', 'B', 'and', 'that', 'if', 'that', 'be', 'the', 'position,', 'the', 'petitioners', 'would', 'be', 'considerably', 'prejudiced', 'by', 

In [194]:
# tokens = ['In',
#  'the',
#  'first',
#  'instance,',
#  'Mr.A.D.Desai',
#  'contended',
#  'that',
#  'the',
#  'petitioners',
#  'might',
#  'not',
#  'have',
#  'led',
#  'full',
#  'evidence',
#  'before',
#  'the',
#  'District',
#  'Court,',
#  'knowing',
#  'or',
#  'at',
#  'least',
#  'believing',
#  'that',
#  'the',
#  'proceedings',
#  'before',
#  'the',
#  'District',
#  'Court',
#  'were',
#  'illegal',
#  'by',
#  'reason',
#  'of',
#  'non-compliance',
#  'with',
#  'the',
#  'requirements',
#  'of',
#  'Section',
#  '56B',
#  'and',
#  'that',
#  'if',
#  'that',
#  'be',
#  'the',
#  'position,',
#  'the',
#  'petitioners',
#  'would',
#  'be',
#  'considerably',
#  'prejudiced',
#  'by',
#  'refusal',
#  'to',
#  'interfere',
#  'with',
#  'the',
#  'order',
#  'of',
#  'the',
#  'District',
#  'Court',
#  'in',
#  'revision.']

# annotations = [{'value':{"start": 26, "end": 35, "text": "A.D.Desai", "labels": ["COURT"]}}, {"value": {"start": 270, "end": 281, "text": "Section 56B", "labels": ["COURT"]}}]
# entity = {"start": 26, "end": 35, "text": "A.D.Desai", "labels": ["COURT"]}

# Generating NER TAGS

In [195]:
bad_rows = []
def generate_ner_tags(row):
  tokens = row["tokens"]
  if row['annotations'][0]['result'] == []:
    return [0] * len(tokens)
  sentence = " ".join(tokens)
  sentence+=" ."
  print(sentence)
  annotations = row['annotations'][0]['result']
  # print(tokens)
  added_indexes = 0
  ner_list = []
  try:
    for annotation in annotations:
      # print(annotation)
      entity = annotation['value']
      # print(entity)
      tokens = [i for i in tokens if i != '']
      ner_list.append((entity["text"].split(), entity["labels"]))
      start_ind=sentence.find(entity["text"])
      if sentence[start_ind-1]!=" ":
        # print("START")
        start_index = len(sentence[:entity["start"]].split())
        i=start_index-1 + added_indexes
        st = tokens[i]
        j=st.find(entity["text"])
        tokens[i:i+1] = [tokens[i][:j], tokens[i][j:]]
        added_indexes += 1
      # print(tokens)
      end_ind=sentence.find(entity["text"]) + len(entity["text"]) - 1
      print(sentence[end_ind+1])
      if sentence[end_ind+1]!=" ":
        # print("END")
        end_index = len(sentence[:entity["end"]].split())
        i=end_index-1 + added_indexes
        st = tokens[i]
        st=st[:-1]
        # print(st)
        # print(entity["text"])
        j=len(st)
        # print(i, j)
        # print([tokens[i][:j], tokens[i][j:]])
        # t=tokens.copy()
      #   t[i:i+1] = [t[i][:j], t[i][j:]]
      # print(t)
        tokens[i:i+1] = [tokens[i][:j], tokens[i][j:]]
        added_indexes += 1
    print("========================")
    # print(tokens)
    # print(ner_list)
    ner_tags = [0] * len(tokens)
    for ner, label in ner_list:
      print(ner, label)
      start_index, end_index = find_sub_list(ner, tokens)
      end_index+=1
      # print(start_index, end_index)
      # print(tokens[start_index:end_index])
      # print(entity_list)

      # Set the NER tag to codes for tokens within the start and end indices if the label is in entity list
      for i in range(len(tokens)):
          if i >= start_index and i < end_index and label[0] in entity_list:
              ner_tags[i] = label_encoding_dict[label[0]]
  except:
    bad_rows.append(row['id'])
    print([])
    print([])
    return [], []
  print(tokens)
  print(ner_tags)
  return ner_tags

In [196]:
dataset['ner_tags'] = dataset.apply(generate_ner_tags, axis=1)
# generate_ner_tags(dataset_judgement.iloc[49])

Streaming output truncated to the last 5000 lines.
['Appeal', 'No.', '280', 'of', '1970'] ['CASE_NUMBER']
['Our', 'attention', 'was', 'also', 'drawn', 'to', 'the', 'observations', 'of', 'Mr.', 'Justice', 'Arun', 'K.', 'Mukherjee', 'in', 'a', 'Bench', 'decision', 'of', 'this', 'Court', 'in', 'the', 'case', 'of', 'Hindusthan', 'Motors', 'Ltd.', 'v.', 'T,', 'N.', 'Kaul', 'in', 'Appeal', 'No.', '280', 'of', '1970', 'in', 'which', 'I', 'was', 'also', 'a', 'party.']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 12, 12, 12, 12, 12, 12, 0, 13, 13, 13, 13, 13, 0, 0, 0, 0, 0, 0, 0]
In Mathura Prasad Bajoo Jaiswal & Ors. v. Dossibai N.B. Jeejeebhey, [1970] 3 SCR 830 the Bench consisting of Shah, CJ., Hegde and Grover, JJ. was called upon to consider whether a decree passed without jurisdiction operates res judicata. .
 
,
 
,
['Mathura', 'Prasad', 'Bajoo', 'Jaiswal', '&', 'Ors.', 'v.', 'Dossibai', 'N.B.', 'Jeejeebhey,', '[1970]', '3', 'SCR', '830'] ['PRECEDENT']


In [197]:
len(bad_rows)

941

In [198]:
len(dataset)

9435

In [199]:
dataset = dataset[~dataset.id.isin(bad_rows)]

In [200]:
len(dataset)

8494

In [201]:
dataset

,id,annotations,data,tokens,ner_tags
0,90d9a97c7b7749ec8a4f460fda6f937e,"[{'result': [{'value': {'start': 90, 'end': 10...",\n\n(7) On specific query by the Bench about a...,"[(7), On, specific, query, by, the, Bench, abo...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,a325c57ba5b84c6fa46bee65e6616633,"[{'result': [{'value': {'start': 26, 'end': 30...","He was also asked whether Agya <span class=""hi...","[He, was, also, asked, whether, Agya, <span, c...","[0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 5, ..."
2,33cface31108441080976035ee04d07e,"[{'result': [{'value': {'start': 13, 'end': 25...","\n5.2 CW3 Mr Vijay Mishra , Deputy Manager, H...","[5.2, CW3, Mr, Vijay, Mishra, ,, Deputy, Manag...","[0, 0, 0, 14, 14, 0, 0, 0, 8, 8, 8, 0, 9, 0, 0..."
3,08debe900b51464e9f26b5d1eecae170,[{'result': []}],You are hereby asked not to carry out any cons...,"[You, are, hereby, asked, not, to, carry, out,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,428070222cd940eba78b233829eb28b8,"[{'result': [{'value': {'start': 18, 'end': 43...",The pillion rider T.V. Satyanarayana Murthy al...,"[The, pillion, rider, T.V., Satyanarayana, Mur...","[0, 0, 0, 5, 5, 5, 0, 0, 0]"
...,...,...,...,...,...
9428,6d8b2a36f61e4415a25017707164c8d7,"[{'result': [{'value': {'start': 79, 'end': 89...",The respondent therefore again made a represen...,"[The, respondent, therefore, again, made, a, r...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, ..."
9429,a5303f0b1462494c89a6d0bb57a86166,"[{'result': [{'value': {'start': 94, 'end': 99...",The distinction which has been made between 'm...,"[The, distinction, which, has, been, made, bet...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, ..."
9430,5cb430a6d0e14e57a968f0c43d219682,"[{'result': [{'value': {'start': 202, 'end': 2...",It is prayed in the application that the suit ...,"[It, is, prayed, in, the, application, that, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9431,6e9147c9e6344865a8b095fb7f43f7c2,"[{'result': [{'value': {'start': 26, 'end': 35...","In the first instance, Mr.A.D.Desai contended ...","[In, the, first, instance,, Mr.A.D.Desai, cont...","[0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


# Formatting Tokens Column

In [202]:
def generate_correct_tokens(row):
  if row['annotations'][0]['result'] == []:
    return row["tokens"]
  print(row['data'])
  tokens = row["tokens"]
  sentence = " ".join(tokens)
  sentence+=" ."
  print(sentence)
  annotations = row['annotations'][0]['result']
  print(tokens)
  added_indexes = 0
  ner_list = []
  for annotation in annotations:
    print(annotation)
    entity = annotation['value']
    print(entity)
    tokens = [i for i in tokens if i != '']
    ner_list.append((entity["text"].split(), entity["labels"]))
    start_ind=sentence.find(entity["text"])
    if sentence[start_ind-1]!=" ":
      print("START")
      start_index = len(sentence[:entity["start"]].split())
      i=start_index-1 + added_indexes
      st = tokens[i]
      j=st.find(entity["text"])
      tokens[i:i+1] = [tokens[i][:j], tokens[i][j:]]
      added_indexes += 1
    # print(tokens)
    end_ind=sentence.find(entity["text"]) + len(entity["text"]) - 1
    print(sentence[end_ind+1])
    if sentence[end_ind+1]!=" ":
      print("END")
      end_index = len(sentence[:entity["end"]].split())
      i=end_index-1 + added_indexes
      print(tokens)
      st = tokens[i]
      st=st[:-1]
      print(st)
      # print(entity["text"])
      j=len(st)
      # print(i, j)
      # print([tokens[i][:j], tokens[i][j:]])
      # t=tokens.copy()
    #   t[i:i+1] = [t[i][:j], t[i][j:]]
    # print(t)
      tokens[i:i+1] = [tokens[i][:j], tokens[i][j:]]
      added_indexes += 1
  print("========================")
  return tokens
  

In [203]:
dataset['tokens'] = dataset.apply(generate_correct_tokens, axis=1)

Streaming output truncated to the last 5000 lines.
According to PW6 ASI Gajender Singh, the second Investigating Officer, on having reached the spot i.e. in front of Guru Nanak Eye Hospital, he arrested the accused. .
['According', 'to', 'PW6', 'ASI', 'Gajender', 'Singh,', 'the', 'second', 'Investigating', 'Officer,', 'on', 'having', 'reached', 'the', 'spot', 'i.e.', 'in', 'front', 'of', 'Guru', 'Nanak', 'Eye', 'Hospital,', 'he', 'arrested', 'the', 'accused.']
{'value': {'start': 21, 'end': 35, 'text': 'Gajender Singh', 'labels': ['WITNESS']}, 'id': 'SCI2GBUV', 'from_name': 'label', 'to_name': 'text', 'type': 'labels'}
{'start': 21, 'end': 35, 'text': 'Gajender Singh', 'labels': ['WITNESS']}
,
END
['According', 'to', 'PW6', 'ASI', 'Gajender', 'Singh,', 'the', 'second', 'Investigating', 'Officer,', 'on', 'having', 'reached', 'the', 'spot', 'i.e.', 'in', 'front', 'of', 'Guru', 'Nanak', 'Eye', 'Hospital,', 'he', 'arrested', 'the', 'accused.']
Singh
{'value': {'start': 115, 'end': 138, 'te

<ipython-input-203-3835ec7ccc07>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['tokens'] = dataset.apply(generate_correct_tokens, axis=1)


In [204]:
dataset

,id,annotations,data,tokens,ner_tags
0,90d9a97c7b7749ec8a4f460fda6f937e,"[{'result': [{'value': {'start': 90, 'end': 10...",\n\n(7) On specific query by the Bench about a...,"[(7), On, specific, query, by, the, Bench, abo...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,a325c57ba5b84c6fa46bee65e6616633,"[{'result': [{'value': {'start': 26, 'end': 30...","He was also asked whether Agya <span class=""hi...","[He, was, also, asked, whether, Agya, <span, c...","[0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 5, ..."
2,33cface31108441080976035ee04d07e,"[{'result': [{'value': {'start': 13, 'end': 25...","\n5.2 CW3 Mr Vijay Mishra , Deputy Manager, H...","[5.2, CW3, Mr, Vijay, Mishra, ,, Deputy, Manag...","[0, 0, 0, 14, 14, 0, 0, 0, 8, 8, 8, 0, 9, 0, 0..."
3,08debe900b51464e9f26b5d1eecae170,[{'result': []}],You are hereby asked not to carry out any cons...,"[You, are, hereby, asked, not, to, carry, out,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,428070222cd940eba78b233829eb28b8,"[{'result': [{'value': {'start': 18, 'end': 43...",The pillion rider T.V. Satyanarayana Murthy al...,"[The, pillion, rider, T.V., Satyanarayana, Mur...","[0, 0, 0, 5, 5, 5, 0, 0, 0]"
...,...,...,...,...,...
9428,6d8b2a36f61e4415a25017707164c8d7,"[{'result': [{'value': {'start': 79, 'end': 89...",The respondent therefore again made a represen...,"[The, respondent, therefore, again, made, a, r...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, ..."
9429,a5303f0b1462494c89a6d0bb57a86166,"[{'result': [{'value': {'start': 94, 'end': 99...",The distinction which has been made between 'm...,"[The, distinction, which, has, been, made, bet...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, ..."
9430,5cb430a6d0e14e57a968f0c43d219682,"[{'result': [{'value': {'start': 202, 'end': 2...",It is prayed in the application that the suit ...,"[It, is, prayed, in, the, application, that, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9431,6e9147c9e6344865a8b095fb7f43f7c2,"[{'result': [{'value': {'start': 26, 'end': 35...","In the first instance, Mr.A.D.Desai contended ...","[In, the, first, instance,, Mr., A.D.Desai, co...","[0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [205]:
dataset = dataset.drop(columns=['annotations', 'data'])

In [206]:
dataset = dataset[dataset['tokens'].map(lambda d: len(d)) > 0]

# Final Dataset Format

In [207]:
dataset

,id,tokens,ner_tags
0,90d9a97c7b7749ec8a4f460fda6f937e,"[(7), On, specific, query, by, the, Bench, abo...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,a325c57ba5b84c6fa46bee65e6616633,"[He, was, also, asked, whether, Agya, <span, c...","[0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 5, ..."
2,33cface31108441080976035ee04d07e,"[5.2, CW3, Mr, Vijay, Mishra, ,, Deputy, Manag...","[0, 0, 0, 14, 14, 0, 0, 0, 8, 8, 8, 0, 9, 0, 0..."
3,08debe900b51464e9f26b5d1eecae170,"[You, are, hereby, asked, not, to, carry, out,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,428070222cd940eba78b233829eb28b8,"[The, pillion, rider, T.V., Satyanarayana, Mur...","[0, 0, 0, 5, 5, 5, 0, 0, 0]"
...,...,...,...
9428,6d8b2a36f61e4415a25017707164c8d7,"[The, respondent, therefore, again, made, a, r...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, ..."
9429,a5303f0b1462494c89a6d0bb57a86166,"[The, distinction, which, has, been, made, bet...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, ..."
9430,5cb430a6d0e14e57a968f0c43d219682,"[It, is, prayed, in, the, application, that, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9431,6e9147c9e6344865a8b095fb7f43f7c2,"[In, the, first, instance,, Mr., A.D.Desai, co...","[0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [208]:
dataset.to_json(r'NER_TRAIN_JUDGEMENT_PREPROCESSED.json')